In [23]:
import sys
import subprocess
import pkg_resources

required = {'sentence-transformers','pandas'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

In [8]:
from sentence_transformers import SentenceTransformer
models_path = "../app/models"
embeddings_path = "../app/embeddings"

# Récupération du modèle non fine-tuné
model_1 = SentenceTransformer(f"{models_path}/sentence_similarity_model")

No sentence-transformers model found with name ../app/models/sentence_similarity_model. Creating a new one with MEAN pooling.


OSError: It looks like the config file at '../app/models/sentence_similarity_model' is not a valid JSON file.

In [11]:
import pickle

with open(f"{models_path}/sentence_similarity_model", "rb") as f:
    model_1 = pickle.load(f)

with open(f"{embeddings_path}/embeddings_corpus_movie", "rb") as f:
    embeddings = pickle.load(f)


In [38]:
embeddings[0]

(array([-7.36547336e-02, -8.33716523e-03,  2.25017332e-02,  1.21309720e-02,
        -2.94153690e-02, -1.64633279e-03,  9.12407339e-02, -7.79220909e-02,
        -8.19904322e-04, -6.69526756e-02, -3.97555484e-03,  1.16377622e-01,
         2.04769801e-03,  4.15166747e-03, -5.63682690e-02, -5.42286560e-02,
         8.86014998e-02, -3.58733051e-02,  3.63019854e-02, -1.30092399e-03,
        -2.36679614e-02,  1.66973565e-02,  3.52966934e-02,  6.91928528e-03,
        -5.88442374e-04, -2.42809914e-02,  5.23280799e-02, -5.54762967e-02,
         1.05778659e-02, -3.43284607e-02,  1.72891561e-02,  4.21549343e-02,
        -9.89390351e-03, -1.00955358e-02,  9.02638286e-02,  2.01865379e-02,
         1.17654037e-02,  2.35742336e-04,  6.49118423e-02,  1.04181387e-01,
         3.06899734e-02, -6.29894882e-02, -2.10242402e-02,  2.61254776e-02,
        -7.93564096e-02,  5.52216684e-03, -2.53872033e-02, -3.42436433e-02,
         4.90081348e-02, -5.43660372e-02,  5.62629336e-03,  6.00129589e-02,
         6.9

In [24]:
import pandas as pd
dataset = pd.read_csv("../Data/movie_synopsis.csv")

In [49]:
from sklearn.metrics.pairwise import cosine_distances,euclidean_distances
def give_suggestion(user_input:str, model, k:int=5):
    # on encode l'input de l'user avec le même modèle pour avoir un vecteur
    embed_user = model.encode(user_input)
    stock_similarity = []
    stock_suggest_k = []
    # on calcule les distances cosinus
    for embed,title,syn  in embeddings:
        result = (cosine_distances([embed],[embed_user]),title)
        #result = (euclidean_distances([embed],[embed_user]),title)
        # elles sont stockés dans la liste sous forme de tuple contenant la distance cosinus et le titre
        stock_similarity.append(result)
    # on trie (ordre croissant) pour avoir le synopsis le plus proche de l'input de l'usr   
    ma_liste_triee = sorted(stock_similarity, key=lambda x: x[0])
    # afficher les premiers résultats etc - on peut changer cette partie en fonction de ce qu'on veut afficher 
    #(idem pour le return)
    #for i, (x, y) in enumerate(ma_liste_triee[:5]):
        #print(f"Suggestion {i+1}: {y} - distance {x}")
        #stock_suggest_k.append(y)
    return ma_liste_triee #ou stock_suggest_k ou stock_similarity[:k]

In [50]:
def eval(model):
    value = 0
    for _, title, syn in embeddings[:10]:
        best = give_suggestion(syn, model)[0][1]
        print(f"best : {best}")
        print(f"title : {title}")
        if best == title:
            value += 1
    return value/len(embeddings[:10])
         

In [51]:
eval(model_1)

best : Four Rooms
title : Four Rooms
best : Judgment Night
title : Judgment Night
best : Life in Loops (A Megacities RMX)
title : Life in Loops (A Megacities RMX)
best : Star Wars
title : Star Wars
best : Finding Nemo
title : Finding Nemo
best : Forrest Gump
title : Forrest Gump
best : American Beauty
title : American Beauty
best : Citizen Kane
title : Citizen Kane
best : Dancer in the Dark
title : Dancer in the Dark
best : The Dark
title : The Dark


1.0